In [2]:
import re
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

# image cut

In [3]:
folder = r"D:\data"
data_path = []

label_map = {
    "Bending": 0,
    "Reverse": 1,
    "Stain": 2
}

for label_name in os.listdir(folder):
    label_folder = os.path.join(folder, label_name)
    if os.path.isdir(label_folder):
        for filename in os.listdir(label_folder):
            if label_name != "Good":
                data_path.append((os.path.join(label_folder, filename), label_map[label_name]))

In [4]:
# topLeft center topRight bottomLeft bottomRight
# image size (840, 840)
sin_ticket_list = [
    [146, 218, 846, 918],
    [883, 233, 1583, 933],
    [1627, 242, 2327, 942],
    [1604, 964, 2304, 1664],
    [141, 947, 841, 1647]
]
sinner = 0
weare = 770

In [5]:
Error_sin_ticket_list = {
    'Bending' : [
        [189, 184, 889, 884],
        [935, 251, 1635, 951],
        [1668, 296, 2368, 996],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ],
    "Reverse": [
        [197, 272, 897, 972],
        [890, 305, 1590, 1005],
        [1621, 315, 2321, 1015],
        [1612, 1050, 2312, 1750],
        [148, 1001, 848, 1701]
    ],
    "Stain" : [
        [169, 244, 869, 944],
        [907, 259, 1607, 959],
        [1636, 337, 2336, 1037],
        [0, 0, 0, 0],
        [141, 979, 841, 1679]
    ],
    "Stain4" : [
        [174, 251, 874, 951],
        [894, 281, 1594, 981],
        [1614, 305, 2314, 1005],
        [1588, 1027, 2288, 1727],
        [122, 990, 822, 1690]
    ]
}

In [6]:
output_path = r"D:\datacut"
for make_filae_name in label_map.keys():
    os.makedirs(output_path + "//" + make_filae_name, exist_ok=True)

In [7]:
label_map = {
    0 : "Bending",
    1 : "Reverse",
    2 : "Stain"
}
cut = {
    0 : 0,
    1 : 0,
    2 : 0
}
dont_save = {
    0 : [19, 20],
    1 : [],
    2 : [24]
}
Error_crop_list = {
    0 : [3],
    1 : [1, 2],
    2 : [4]
}
for image_path in tqdm(data_path):
    image = Image.open(image_path[0])
    label = image_path[1]
    for number in range(1, 6):
        cut[label] += 1
        if label == 2 and (cut[label] - 1) // 5 == 1:
            error_crops = Error_sin_ticket_list[label_map[label] + "4"]
            left, upper, right, lower = error_crops[number - 1]
        elif (cut[label] - 1) // 5 in Error_crop_list[label]:
            error_crops = Error_sin_ticket_list[label_map[label]]
            left, upper, right, lower = error_crops[number - 1]
        else:
            left, upper, right, lower = sin_ticket_list[number - 1]
        output_filename = os.path.join(f"{output_path}\\{image_path[0][8:].replace('.bmp', '')}_{number}.jpg")
        output_filename = output_filename.replace(" ", "")
        cropped_region = (left, upper, right, lower)
        cropped_image = image.crop(cropped_region)
        if cut[label] not in dont_save[label]:
            cropped_image.save(output_filename, 'JPEG')

100%|██████████| 12/12 [00:03<00:00,  3.13it/s]
